In [4]:
import os
import numpy as np
import pandas as pd
import sys
# !pip3 install --upgrade --force-reinstall tensorflow-gpu
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
import tensorflow as tf
from tensorflow import keras

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [7]:
os.chdir('/content/drive/My Drive/Colab Notebooks/summarization')
print("We are here: ",os.getcwd())
print(os.listdir())

We are here:  /content/drive/My Drive/Colab Notebooks/summarization
['sent', 'summ', 'dailymail.tgz', 'cnn.tgz', 'data_read_test.ipynb', 'glove.6B.300d.txt', 'summ2', 'model.ipynb']


In [8]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=sys.maxsize)

Using TensorFlow backend.


In [0]:
#Initializing tokenizer for Vocabulary
data1 = open('sent').readlines()
data2 = open('summ2').readlines()

tokenizer.fit_on_texts(data1)
tokenizer.fit_on_texts(data2)


In [10]:
print(len(data2),len(data1))

92454 92454


In [11]:
'''with open('summ2','w') as f:
  bos = "<BOS> "
  eos = " <EOS>"
  for i in data2:
    i = i.strip()
    final = bos + i + eos
    f.write(final + '\n')'''

'with open(\'summ2\',\'w\') as f:\n  bos = "<BOS> "\n  eos = " <EOS>"\n  for i in data2:\n    i = i.strip()\n    final = bos + i + eos\n    f.write(final + \'\n\')'

In [32]:
dictionary = tokenizer.word_index
word2idx = {}
idx2word = {}
num_encoder_tokens = len(tokenizer.word_index)+1
num_decoder_tokens = len(tokenizer.word_index)+1
for k, v in dictionary.items():
    word2idx[k] = v
    idx2word[v] = k
print(idx2word[276545])

sioblooula


In [13]:
#Finding the maximum sequence length
MAX_INPUT_LENGTH = max(len(i.split()) for i in data1)
print(MAX_INPUT_LENGTH)
MAX_TARGET_LENGTH = max(len(j.split()) for j in data2)
print(MAX_TARGET_LENGTH)

2009
112


In [0]:
# Preparing GloVe
EMBEDDING_DIM = 300
embeddings_index = {}
f = open(os.path.join('', 'glove.6B.{}d.txt'.format(EMBEDDING_DIM)))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [35]:
"u.s" in embeddings_index

True

In [0]:
#Embedding matrix
embedding_matrix = np.zeros((len(tokenizer.word_index)+1, EMBEDDING_DIM),dtype='float32')
for word,i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
    # Words not found in glove will be zeros
        embedding_matrix[i] = embedding_vector

In [22]:
print(embedding_matrix.shape)

(330317, 300)


In [0]:
from keras.layers import Embedding
from keras.layers import Dense, LSTM, Input, concatenate
from keras.models import Model
# tf.random_uniform = tf.random.uniform
# tf.get_default_session = tf.compat.v1.get_default_session
batch_size = 64
epochs = 100
HIDDEN_UNITS_ENC = 256
num_samples = 10000

In [24]:
encoder_inputs = Input(shape=(MAX_INPUT_LENGTH,), name='encoder_inputs')
embedding_layer = Embedding(len(tokenizer.word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                            input_length=MAX_INPUT_LENGTH, trainable=False, name='embedding_layer')

encoder_rnn = LSTM(units=HIDDEN_UNITS_ENC, return_state=True, dropout=0.5, recurrent_dropout=0.5,name='encoder_lstm')
encoder_output, state_h_f, state_c_f = encoder_rnn(embedding_layer(encoder_inputs))
encoder_rnn2 = LSTM(units=HIDDEN_UNITS_ENC, return_state=True, dropout=0.5, recurrent_dropout=0.5,
go_backwards=True,name='encoder_lstm_backward')
encoder_output, state_h_b, state_c_b = encoder_rnn2(embedding_layer(encoder_inputs))

state_h = concatenate([state_h_f, state_h_b])
state_c = concatenate([state_c_f, state_c_b])

encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(MAX_TARGET_LENGTH,), name='decoder_inputs')
embedding_layer = Embedding(len(tokenizer.word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
input_length=MAX_TARGET_LENGTH, trainable=False, name='emb_2')
decoder_lstm = LSTM(HIDDEN_UNITS_ENC * 2, return_sequences=False, return_state=True, dropout=0.5,
recurrent_dropout=0.5, name='decoder_lstm')
decoder_outputs, state_h, state_c = decoder_lstm(embedding_layer(decoder_inputs), initial_state=encoder_states)

decoder_dense = Dense(len(tokenizer.word_index) + 1, name='decoder_dense')
decoder_outputs = decoder_dense(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)











Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [25]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])